#**Участники проекта:**
#Блоцкий Александр
#Богородецкий Александр
#Мирошкин Артемий
#Храмков Андрей
#**Ccылка на бот**:
@Hellenic23323_bot
#**API бота**:
6690051177:AAFoLliysecMtXqqKvUT_dLS1C9DWQLkl4k
#**API для нейросети**:
r8_cZs6qtWAspitHrOcPnrlIFy80NWUBXu36gVvt

#**Ссылка на Github** (там содержится небольшой древнегреческий словарь): https://github.com/AleksBLOCKI/Simple_greek_text.git

##Описание
 В процессе работы API от OpenAI не заработал (нет возможности пополнить аккаунт), пришлось использовать условно-бесплатный аналог его. То есть llama-2-70b от Meta.

 Наш Бот работает с небольшим набором данных (словарем для учебника Αθηναζε), и использует технологию RAG (генерация с расширением поиска, англ.  *retrieval-augmented generation*). Эта технология используется в областях обработки естественного языка и машинного обучения, и она прекрасно подходит для нашей задачи. Она включает в себя два основных действия: **поиск** и **генерацию**.
 ### Поиск
 С помощью RAG осуществляется поиск релевантной информации или документов из набора данных (в нашем случае, из словаря). Модель поиска отбирает полезную для генерации овтета информацию, то есть такие данные, которые содержат в себе ответ или контекст для  запроса. Поиск выполняется с использованием векторного поиска, когда и запрос, и документы встраиваются в многомерное векторное пространство, а для поиска наиболее релевантных документов используются показатели сходства между запоросом и имеющимся индексом документов.
 ### Генерация
 Модели генерации языка (в нашем случае это нейросеть llama-2-70b), используя найденную информацию и поисковый запрос генерируют ответ. Ответ, сгенерированный таким образом, более релевантен. RAG имеет доступ к более широкому (и более точному) направлению данных, в отличие от обычной языковой модели. Кроме того, технология позволяет извлечь максимально подходящую информацию из предоставленных данных. По сути, эта технология создаёт контекст, необходимый нейросети для написания ответа.








# Код

Загружаем данные из нашего репозитария GitHub.


In [ ]:
!git clone https://github.com/AleksBLOCKI/Simple_greek_text.git

Cloning into 'Simple_greek_text'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 23 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (23/23), 72.07 KiB | 4.24 MiB/s, done.
Resolving deltas: 100% (2/2), done.


Если словарь по какой-то причине не загрузится то эти строчки кода (испольуя условные конструкции) заново запустят загрузку данных.

In [ ]:
import os
path = '/content/Simple_greek_text/Data/Corrected_Greek_Words_Translations.txt'

if os.path.exists(path):
    path=path
else:
  !git clone https://github.com/AleksBLOCKI/Simple_greek_text.git


Во-первых, устанавливаем нужные библиотеки.

In [ ]:
pip install llama-index transformers accelerate bitsandbytes replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.5/966.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling

Далее из llama-index импортируем нужный нам инструментарий.

In [ ]:
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext,
)


Присваиваем documents значение
**SimpleDirectoryReader** - из llama-index, это средство для чтения каталогов.
Программа выберет лучшую программу чтения файлов на основе расширений файлов.

In [ ]:
documents = SimpleDirectoryReader("/content/Simple_greek_text/Data").load_data()

Добавляем наш API нейросети.


In [ ]:
import os
os.environ['REPLICATE_API_TOKEN'] = 'r8_cZs6qtWAspitHrOcPnrlIFy80NWUBXu36gVvt'

1. устанавливаем client library в нашу программу, чтобы работала нейросеть
2. затем делаем так, чтобы чтобы нейросеть llama считала промты (поле введения текста в программе) и импортировала в нейросеть
3. вписываем ключ нейросети, чтобы активировать ее
4. выставялем нужную версию нейросети
5. выставляет параметр temperature, отвечающий за строгость подбора соответствий в наших библиотеках (словарь)
6. debug true вводится для того, чтобы проверять используемые файлы (взят из инструкции к использованию llama), top_p и top_k нужны для того, чтобы можно определить поле возможных ответов, то есть сделать их более строгими или более случайными. В данном случае используется для того, чтобы результат ответа мог быть больше чем одно слово. Максимальное количество токенов 25 - макимальное количество слов в ответе
7. последующие команды - протокольные для использования инструментария нейросети
8. далее механизм вывода ответа нейросети



In [ ]:
from llama_index.llms import Replicate
from llama_index.prompts import PromptTemplate

llama2_70b_chat = "meta/llama-2-70b:a52e56fee2269a78c9279800ec88898cecb6c8f1df22a6483132bea266648f00"
llm = Replicate( #далее, настраиваем параметры нашей нейросети
    model=llama2_70b_chat,
    temperature=0.70,
    additional_kwargs={"debug": True, "top_p": 0.5, "top_k": 30, "max_new_tokens": 25},
)
from llama_index import set_global_tokenizer # импортируем токенизатор
from transformers import AutoTokenizer
from llama_index import ServiceContext

set_global_tokenizer(
    AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf").encode
)
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Создание индекса
Создадим наш индекс словаря с помощью VectorStoreIndex. Это – ключевой процесс в генерайии с расширением поиска (RAG).
Векторные хранилища принимают список объектов (в нашем случае это *documents*), и составляют из них индекс.
Документ разбивается на фрагменты и поверх каждого из фрагментов добавляются абстракции, то есть такие значения, которые отслеживают метаданные документа и его взаимосвязи. Традиционные методы (линейный поиск) слишком медленные для большого массива данных, поэтому мы используем Векторный индекс для быстрого и точного поиска.

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

index = VectorStoreIndex.from_documents( #создаём индекс из нашего словаря
    documents, service_context=service_context ) #service_context - это набор необходимых для индексации ресурсов

# Логи

In [ ]:
def logs(filename, content): #в параметр filename запишется имя файла, в content - содержание лога
    if not isinstance(content, str): # на этом этапе мы определяем: является ли содержание ли содеражние строкой или нет? если нет - то оно преобразуется в нее.
        content = str(content)

    with open(filename, 'a', encoding='utf-16') as file: #открытие filename с кодировкой utf-16. после выполнения команда with, он закроется.
        file.write(content + "\n") #записываем содержание лога в filename, после чего добавляем символ переноса строки.

# Получение запроса и обработка. Вывод результатов

In [ ]:
import re

def process_input():
    while True:
        user_input = input("Введите текст для перевода: ")
        if user_input.strip():
            words = user_input.split()

            validgreek = re.match(r'^[\u0370-\u03FF\u1F00-\u1FFF\s]+$', user_input) #validgreek ограничивает ввод только греческими символами (в кодировке UTF это [\u0370-\u03FF\u1F00-\u1FFF\s)
            wordlenok = all(len(word) <= 30 for word in words) #wordlenok ограничивает ввод только 30 символами
            minoneword = len(words) >= 1 #minoneword ограничивает ввод как минимум одним словом
            nostartspace = not user_input.startswith(' ') #nostartspace запрещаёт начинать ввод с пробела

            if validgreek and (wordlenok or minoneword) and nostartspace:
                return user_input
            else:
                messages = [] #В случае ошибок, программа выводит нужное сообщение
                if not validgreek:
                    messages.append("Пожалуйста, вводите только греческие буквы.")
                if not (wordlenok or minoneword):
                    messages.append("Не вводите больше 30 символов.")
                if not nostartspace:
                    messages.append("Слово не может начинаться с пробела.")

                print(" ".join(messages))
        else:
            print("Поле не может быть пустым.")

user_input = process_input()


Введите текст для перевода: ἀγών


In [ ]:
query_engine = index.as_query_engine(response_mode="compact") #Здесь осуществляется поиск по нашему индексу
response = query_engine.query(f"переведи на русский: {user_input}") #Выдаём запрос нейросети, response присваиваем значение полученного ответа
logs('/content/answers.txt', response) #этот код мы используем для записи логов в файл '/content/answers.txt'

In [ ]:
response_str = str(response) #Ответы не всегда могут быть подходящими по своей структуре, поэтому мы объединяем их в одну линию, а затем разбиваем на несколько строк
line = response_str.split('\n')[0] #Ответом будет только первая строка

print(line)

ἀγών = состязания


# Телеграм-бот


In [ ]:
pip install pytelegrambotapi #библиотека для загрузки бота в питон

In [ ]:
import telebot
import re # импортируем модуль под названием "re", который предоставляет функции для работы с регулярными выражениями

In [ ]:
TOKEN = '6690051177:AAFoLliysecMtXqqKvUT_dLS1C9DWQLkl4k' #вводим привязанный к боту токен
bot = telebot.TeleBot(TOKEN)

In [ ]:
  def process_greek_text(text):
      messages = [] #создается пустой список для хранения сообщений об ошибках
      if text.strip():
          words = text.split() #разбиваем текст на слова
          validgreek = re.match(r'^[\u0370-\u03FF\u1F00-\u1FFF\s]+$', text) #проверка на греческие буквы
          wordlenok = all(len(word) <= 30 for word in words) #верхний предел символов
          minoneword = len(words) >= 1         #нижний предел слов
          nostartspace = not text.startswith(' ') #проверяем на отстутствие пробелов в начале

          if validgreek and (wordlenok or minoneword) and nostartspace:
              query_engine = index.as_query_engine(response_mode="compact")
              response = query_engine.query(f"переведи на русский: {text}")
              logs('/content/answers.txt', response)
              response_str = str(response)
              line = response_str.split('\n')[0]
              query_response = f"Для {text} перевод: {line}"
              return query_response # Если все проверки прошли успешно, то выполняется запрос к поисковому движку, результат записывается в файл и возвращается строка с переводом
          else:
              if not validgreek:
                  messages.append("Пожалуйста, вводите только греческие буквы.")
              if not (wordlenok or minoneword):
                  messages.append("Не вводите больше 30 символов.")
              if not nostartspace:
                  messages.append("Слово не может начинаться с пробела.")
      else:
          messages.append("Поле не может быть пустым.")

      return '\n'.join(messages) #возвращается строка, состоящая из всех сообщений об ошибках, разделенных символом новой строки

@bot.message_handler(func=lambda message: True) #декоратор, который позволяет получать сообщения из бота и их обрабатывать
def handle_message(message): #определение функции обработки сообщений
    user_input = message.text #извлекаем текст сообщения из объекта message и сохраняем его в переменную user_input
    response = process_greek_text(user_input) #вызываем функцию process_greek_text, передавая ей текст пользователя. Функция обрабатывает текст и возвращает результат. Результат сохраняется в переменную response

    bot.send_message(message.chat.id, response) #бот отправляет ответ пользователю

bot.polling() #метод библиотеки pyTelegramBotAPI для Python, который используется для непрерывного получения новых обновлений от Telegram сервера
